# DeepSeek OCR API Server for Google Colab (Cloudflare Tunnel版)

このノートブックはGoogle Colab上でDeepSeek OCRをAPIサーバーとして実行します。

## Cloudflare Tunnelの利点
- ✅ **認証トークン不要**（ngrokより簡単）
- ✅ **完全無料**
- ✅ **安定性が高い**
- ✅ **セットアップが簡単**

## 使い方
1. ランタイム → ランタイムのタイプを変更 → **GPU** を選択
2. すべてのセルを順番に実行
3. 表示されたCloudflare URLをコピーして、MacBook Proの`.env`に設定

## 必要なもの
- Google アカウントのみ（追加登録不要！）


## 1. GPU確認

In [ ]:
!nvidia-smi

## 2. Cloudflaredのインストール

In [ ]:
# Cloudflaredのインストール
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb
!rm cloudflared-linux-amd64.deb

# バージョン確認
!cloudflared --version

print("✅ Cloudflaredインストール完了")

## 3. 依存関係のインストール

In [ ]:
# DeepSeek OCR依存関係
!pip install -q torch torchvision torchaudio
!pip install -q transformers==4.46.3
!pip install -q tokenizers==0.20.3
!pip install -q einops addict easydict
!pip install -q Pillow PyYAML opencv-python-headless
!pip install -q flask flask-cors

# Flash Attention（オプション、高速化）
!pip install -q flash-attn==2.7.3 --no-build-isolation || echo "Flash Attention インストール失敗（スキップ）"

print("✅ 依存関係インストール完了")

## 4. DeepSeek OCRプロセッサーの実装

In [ ]:
import os
import sys
import yaml
import re
from pathlib import Path
import torch
from transformers import AutoModel, AutoTokenizer
from PIL import Image
import io
import base64

class DeepSeekOCRProcessor:
    """DeepSeek-OCRを使用してLP画像を処理"""

    def __init__(self, model_name='deepseek-ai/DeepSeek-OCR'):
        print(f"Loading DeepSeek-OCR model: {model_name}")
        print("初回は5-10分かかります。しばらくお待ちください...")
        
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )

        self.model = AutoModel.from_pretrained(
            model_name,
            _attn_implementation='flash_attention_2',
            trust_remote_code=True,
            use_safetensors=True
        )

        self.model = self.model.eval().cuda().to(torch.bfloat16)
        print("✅ Model loaded successfully")

    def process_image_to_yaml(self, image_data, output_path='/tmp/deepseek_output'):
        """
        画像データ（base64またはバイト）からYAMLを生成
        """
        # 一時ディレクトリの作成
        os.makedirs(output_path, exist_ok=True)
        temp_image_path = os.path.join(output_path, 'temp_image.png')
        
        # 画像データを保存
        if isinstance(image_data, str):
            # base64デコード
            image_bytes = base64.b64decode(image_data)
            with open(temp_image_path, 'wb') as f:
                f.write(image_bytes)
        else:
            # バイトデータ
            with open(temp_image_path, 'wb') as f:
                f.write(image_data)
        
        # DeepSeek OCR実行
        prompt = "<image>\n<|grounding|>Convert the document to markdown. "
        
        result = self.model.infer(
            self.tokenizer,
            prompt=prompt,
            image_file=temp_image_path,
            output_path=output_path,
            base_size=1024,
            image_size=640,
            crop_mode=True,
            save_results=False,
            test_compress=True
        )
        
        # Markdownを取得
        if isinstance(result, dict):
            markdown_text = result.get('text', '') or result.get('content', '')
        else:
            markdown_text = str(result)
        
        # Markdown → YAML変換
        yaml_text = self.markdown_to_yaml(markdown_text)
        
        return {
            'yaml': yaml_text,
            'markdown': markdown_text
        }

    def markdown_to_yaml(self, markdown_text):
        """Markdown → YAML変換（簡易版）"""
        sections = []
        lines = markdown_text.split('\n')
        current_section = None
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            if line.startswith('#'):
                if current_section:
                    sections.append(current_section)
                
                heading_text = line.lstrip('#').strip()
                current_section = {
                    'type': 'content',
                    'texts': [{'content': heading_text, 'role': 'headline'}],
                    'buttons': [],
                    'items': []
                }
            elif current_section:
                current_section['texts'].append({'content': line, 'role': 'body'})
        
        if current_section:
            sections.append(current_section)
        
        if not sections:
            sections.append({
                'type': 'content',
                'texts': [{'content': markdown_text[:500], 'role': 'body'}],
                'buttons': [],
                'items': []
            })
        
        yaml_data = {
            'meta': {
                'generator': 'DeepSeek-OCR',
                'template_version': '2.0'
            },
            'sections': {f'section{i+1}': s for i, s in enumerate(sections)}
        }
        
        return yaml.dump(yaml_data, allow_unicode=True, default_flow_style=False)

# グローバルインスタンス（初回ロード時のみ）
processor = None

def get_processor():
    global processor
    if processor is None:
        processor = DeepSeekOCRProcessor()
    return processor

print("✅ DeepSeekOCRProcessor定義完了")

## 5. Flask APIサーバーの実装

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import time

app = Flask(__name__)
CORS(app)  # CORS有効化

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        'status': 'ok',
        'service': 'DeepSeek-OCR API Server (Cloudflare Tunnel)',
        'gpu': torch.cuda.is_available(),
        'gpu_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else None
    })

@app.route('/ocr', methods=['POST'])
def ocr_endpoint():
    start_time = time.time()
    
    try:
        # ファイルアップロードまたはbase64データ
        if 'file' in request.files:
            file = request.files['file']
            image_data = file.read()
        elif request.is_json and 'image' in request.json:
            image_data = request.json['image']  # base64
        else:
            return jsonify({'success': False, 'error': 'No image provided'}), 400
        
        # DeepSeek OCR実行
        proc = get_processor()
        result = proc.process_image_to_yaml(image_data)
        
        processing_time = time.time() - start_time
        
        return jsonify({
            'success': True,
            'yaml': result['yaml'],
            'markdown': result['markdown'],
            'processingTime': processing_time * 1000,  # ms
            'metadata': {
                'modelType': 'DeepSeek-OCR',
                'processingTime': processing_time * 1000
            }
        })
    
    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

print("✅ Flask APIサーバー定義完了")

## 6. Cloudflare Tunnelの起動

**認証トークン不要！** このセルを実行するだけでOKです。

In [ ]:
import threading
import subprocess
import time
import re

# モデルを事前ロード
print("\n" + "="*60)
print("📦 DeepSeek-OCRモデルをロード中...")
print("初回は5-10分かかります。コーヒーでも飲んでお待ちください☕")
print("="*60 + "\n")

get_processor()

print("\n✅ モデルロード完了")
print("\n" + "="*60)
print("🚀 APIサーバーを起動中...")
print("="*60 + "\n")

# Flaskサーバーをバックグラウンドで起動
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Flaskが起動するまで待機
time.sleep(3)

# Cloudflare Tunnelを起動
print("🌐 Cloudflare Tunnelを起動中...\n")

cloudflared_process = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Cloudflare URLを取得
public_url = None
for line in cloudflared_process.stdout:
    print(line.strip())
    
    # URLを抽出
    if 'trycloudflare.com' in line or 'https://' in line:
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
        if match:
            public_url = match.group(0)
            break

if public_url:
    print("\n" + "="*60)
    print("🎉 DeepSeek OCR APIサーバー起動完了！")
    print("="*60)
    print(f"\n📡 公開URL: {public_url}")
    print(f"\n✅ ヘルスチェック: {public_url}/health")
    print(f"✅ OCRエンドポイント: {public_url}/ocr")
    print("\n" + "-"*60)
    print("💡 MacBook Proでの設定方法:")
    print("-"*60)
    print("\n1. .envファイルに以下を追加:")
    print(f"   DEEPSEEK_COLAB_URL={public_url}")
    print("\n2. アプリケーションを起動:")
    print("   npm run dev")
    print("\n3. http://localhost:3000 にアクセス")
    print("\n" + "="*60)
    print("⚠️  重要な注意事項")
    print("="*60)
    print("• このセルを実行し続けている限りサーバーは稼働します")
    print("• Colabセッションが切れたら再起動が必要です（無料版:12時間）")
    print("• URLはセッションごとに変わります")
    print("="*60 + "\n")
    
    # サーバーを稼働し続ける
    try:
        cloudflared_process.wait()
    except KeyboardInterrupt:
        print("\n⏹️  サーバーを停止しました")
        cloudflared_process.terminate()
else:
    print("\n⚠️ Cloudflare URLの取得に失敗しました")
    print("上記のログを確認してください")

## テスト（オプション）

別のセルで、APIが正常に動作しているかテストできます。

In [ ]:
import requests

# 上で表示された公開URLを使用
API_URL = "YOUR_CLOUDFLARE_URL"  # 例: https://xxxx.trycloudflare.com

if API_URL != "YOUR_CLOUDFLARE_URL":
    # ヘルスチェック
    try:
        response = requests.get(f"{API_URL}/health", timeout=10)
        print("✅ ヘルスチェック成功:")
        print(response.json())
    except Exception as e:
        print(f"❌ ヘルスチェック失敗: {e}")
else:
    print("⚠️ API_URLを設定してください")